In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import  XGBClassifier
from xgboost import plot_importance
import xgboost
from sklearn import metrics
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus'] = False
if platform.system() == 'Darwin':    # 맥
    rc( 'font', family='AppleGothic' )
elif platform.system() == 'Windows': # 윈도우
    fontPath = 'c:/Windows/Fonts/malgun.ttf'
    fontName = font_manager.FontProperties( fname=fontPath ).get_name()
    rc( 'font', family=fontName )
else:
    print('알수없는 시스템. 미적용')
    
pd.set_option('display.expand_frame_repr', False)
import warnings
warnings.filterwarnings(action='ignore')

# Selection


In [2]:
final_dataset = pd.read_excel('./true.xlsx')

col = ['종가 (5일 평균)(원)_1일변화율',
       '변동성 (5일)_1일변화율', '순매수수량(외국인계)(20일합산)(주)_1일변화율', 'PER(IFRS-연결)_1일변화율',
       '순매수수량(개인)(20일합산)(주)_1일변화율', '종가 (60일 평균)(원)_1일변화율',
       '거래량 (20일 평균)(주)_1일변화율', '순매수수량(외국인계)(60일합산)(주)_1일변화율',
       '순매수수량(기관/외국인계)(60일합산)(주)_1일변화율', '순매수수량(외국인계)(주)_1일변화율',
       '순매수수량(기관계)(주)_1일변화율', '순매수수량(기관계)(60일합산)(주)_1일변화율',
       '순매수수량(개인)(주)_1일변화율', '외국인보유비중(티커)(%)_1일변화율', 'PSR(IFRS-연결)_1일변화율',
       '종가 (120일 평균)(원)_1일변화율', '종가 (20일 평균)(원)_1일변화율',
       '순매수수량(개인)(60일합산)(주)_1일변화율', '거래량 (60일 평균)(주)_1일변화율',
       'PCR(IFRS-연결)_1일변화율', '순매수수량(기관/외국인계)(20일합산)(주)_1일변화율',
       '순매수수량(기관계)(20일합산)(주)_1일변화율', '수익률 (1주)(%)_1일변화율', 'PBR(IFRS-연결)_1일변화율',
       '거래량 (5일 평균)(주)_1일변화율', '순매수수량(기관/외국인계)(주)_1일변화율',
       '종가 (5일 평균)(원)_2일변화율', '변동성 (5일)_2일변화율', '순매수수량(외국인계)(20일합산)(주)_2일변화율',
       'PER(IFRS-연결)_2일변화율', '순매수수량(개인)(20일합산)(주)_2일변화율',
       '종가 (60일 평균)(원)_2일변화율', '거래량 (20일 평균)(주)_2일변화율',
       '순매수수량(외국인계)(60일합산)(주)_2일변화율', '순매수수량(기관/외국인계)(60일합산)(주)_2일변화율',
       '순매수수량(외국인계)(주)_2일변화율', '순매수수량(기관계)(주)_2일변화율',
       '순매수수량(기관계)(60일합산)(주)_2일변화율', '순매수수량(개인)(주)_2일변화율',
       '외국인보유비중(티커)(%)_2일변화율', 'PSR(IFRS-연결)_2일변화율', '종가 (120일 평균)(원)_2일변화율',
       '종가 (20일 평균)(원)_2일변화율', '순매수수량(개인)(60일합산)(주)_2일변화율',
       '거래량 (60일 평균)(주)_2일변화율', 'PCR(IFRS-연결)_2일변화율',
       '순매수수량(기관/외국인계)(20일합산)(주)_2일변화율', '순매수수량(기관계)(20일합산)(주)_2일변화율',
       '수익률 (1주)(%)_2일변화율', 'PBR(IFRS-연결)_2일변화율', '거래량 (5일 평균)(주)_2일변화율',
       '순매수수량(기관/외국인계)(주)_2일변화율','수익률(%)(2)','Name']
data =  final_dataset[col]

In [3]:
len(col)

54

In [4]:
def data_sp(data):
    com_name_set = list(set(data.Name))
    
    train_set = pd.DataFrame()
    test_set = pd.DataFrame()
    
    for i in range(len(com_name_set)):
        sub_set = data[ data.Name == com_name_set[i]]
        train_set = train_set.append(sub_set[:-60])
        test_set = test_set.append(sub_set[-60:])
    return train_set , test_set

In [5]:
train, test = data_sp(data)

In [6]:
print(train.shape,test.shape)

(1305, 54) (60, 54)


### 파라미터 튜닝
파라미터 튜닝을 해주는 모듈이 있는데 그 모듈을 사용할 경우 일반적인 교차검증을 한 상태로 결과를 비교하기에 쓸수가 없었다.
그래서 직접 파라미터 값들을 넣고 줄여나가는 식으로 하였다.

In [7]:
col = ['순매수수량(외국인계)(60일합산)(주)_1일변화율',
 '순매수수량(기관/외국인계)(60일합산)(주)_1일변화율',
 '순매수수량(외국인계)(주)_1일변화율',
 '순매수수량(기관계)(주)_1일변화율',
 '종가 (120일 평균)(원)_1일변화율',
 '종가 (20일 평균)(원)_1일변화율',
 '순매수수량(기관계)(20일합산)(주)_1일변화율',
 '거래량 (5일 평균)(주)_1일변화율',
 '순매수수량(기관/외국인계)(주)_1일변화율',
 'PER(IFRS-연결)_2일변화율',
 '종가 (60일 평균)(원)_2일변화율',
 '순매수수량(외국인계)(60일합산)(주)_2일변화율',
 '거래량 (60일 평균)(주)_2일변화율',
 '수익률 (1주)(%)_2일변화율',
 '거래량 (5일 평균)(주)_2일변화율']
X_train = train[col]
X_test = test[col]

Y_train = train['수익률(%)(2)']
Y_test = test['수익률(%)(2)']

In [8]:
from sklearn import metrics

In [9]:
backtestdata = pd.read_excel('./act.xlsx')

In [45]:
val_acc = []
acc = []
pre= []
param = []
for i in [4,5,6]:
    print(i)
    for b in [0.01,0.05, 0.1]:
        for c in [0,1,2,]:
            for d in [0.5,1,2,3]:
                for f in [0.8,0.9,1]:
                    for g in [100]:
                        model = XGBClassifier(max_depth=i, 
                                              learning_rate=b,
                                              n_estimators = g,
                                              gamma = c,
                                              min_child_weight=d,
                                              colsample_bytree= f,
                                              n_jobs=-1)

                        model.fit(X_train ,Y_train)
                        j = model.predict(X_train)
                        a = model.predict(X_test)
                        t = metrics.confusion_matrix(Y_test,a)
                        
                        val_acc.append(metrics.accuracy_score(j, Y_train))
                        acc.append(metrics.accuracy_score(a,Y_test))
                        pre.append(t[1,1]/t[:,1].sum())
                        param.append(model)

result = pd.concat([pd.DataFrame(val_acc),pd.DataFrame(acc),abs(pd.DataFrame(val_acc) - pd.DataFrame(acc)),pd.DataFrame(pre),pd.DataFrame(param)],axis=1)
result.columns = ['val_acc','acc','mi','pre','paremeter']

4
5
6


In [46]:
result.to_excel('./tuning_result1.xlsx',encoding = 'utf-8')

#result2

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=1, learning_rate=0.05, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)
       
val / test 차이가 적고 성능이 좋아서 이 파라미터를 기준으로 파라미터 서칭

In [50]:
acc = []
pre= []
param = []
for i in [5]:
    print(i)
    for b in np.linspace(0.03,0.07,5):
        for c in np.linspace(0.8, 1.2, 5):
            for d in np.linspace(0.8, 1.2, 5):
                for f in [0.85, 0.9, 0.95]:
                    for g in [100]:
                        model = XGBClassifier(max_depth=i, 
                                              learning_rate=b,
                                              n_estimators = g,
                                              gamma = c,
                                              min_child_weight=d,
                                              colsample_bytree= f,
                                              n_jobs=-1)

                        model.fit(X_train ,Y_train)
                        a = model.predict(X_test)
                        t = metrics.confusion_matrix(Y_test,a)

                        val_acc.append(metrics.accuracy_score(j, Y_train))
                        acc.append(metrics.accuracy_score(a,Y_test))
                        pre.append(t[1,1]/t[:,1].sum())
                        param.append(model)

result2 = pd.concat([pd.DataFrame(val_acc),pd.DataFrame(acc),abs(pd.DataFrame(val_acc) - pd.DataFrame(acc)),pd.DataFrame(pre),pd.DataFrame(param)],axis=1)
result2.columns = ['val_acc','acc','mi','pre','paremeter']

5


In [53]:
result2.to_excel('./tuning_result2.xlsx',encoding = 'utf-8')

# 결과 
acc가 가장 높은 모델을 봤을 때 val_acc와 acc의 차이가 6%로 차이가 심했다.   
그래서 val_acc와 acc의 차이가 1% 미만이고 성능 또한 우수한 모델을 찾았다.
acc가 같은 경우 precision순으로 뽑았다.

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=1.2, learning_rate=0.07,
       max_delta_step=0, max_depth=5, min_child_weight=1.0, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)
       
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=1.2, learning_rate=0.07,
       max_delta_step=0, max_depth=5, min_child_weight=1.1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)
       
2개의 모델이 나왔다.  min_child_weight가 0.1 차이 났다.

In [66]:
acc4 = []
pre4= []
param4 = []
# 7은 상위권에 없었다. 
for d in [100,300,500,800,1000,1500,2000,2500,3000,4000,5000]:
    XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
   colsample_bytree=0.95, gamma=0.9, learning_rate=0.07,
   max_delta_step=0, max_depth=5, min_child_weight=1.1, missing=None,
   n_estimators=d, n_jobs=-1, nthread=None,
   objective='binary:logistic', random_state=0, reg_alpha=0,
   reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
   subsample=1)

    model.fit(X_train ,Y_train)
    a = model.predict(X_test)
    t = metrics.confusion_matrix(Y_test,a)

    acc4.append(metrics.accuracy_score(a,Y_test))
    pre4.append(t[1,1]/t[:,1].sum())
    param4.append(d)

result4 = pd.concat([pd.DataFrame(acc4),pd.DataFrame(pre4),pd.DataFrame(param4)],axis=1)
result4.columns = ['acc','pre','paremeter']
result4

,acc,pre,paremeter
0,0.7,0.689655,100
1,0.7,0.689655,300
2,0.7,0.689655,500
3,0.7,0.689655,800
4,0.7,0.689655,1000
5,0.7,0.689655,1500
6,0.7,0.689655,2000
7,0.7,0.689655,2500
8,0.7,0.689655,3000
9,0.7,0.689655,4000


n_estimators에 의한 성능차이는 없었다 이는 적은 수의 트리로만으로도 분석에 충분하다는 의미다

## Back testing
- 필자는 HP노트북인데 같은 random_state를 주어도 나 이외의 컴퓨터와는 다른 결과를 얻는 신기한(?) 경험을 하였다.

In [22]:
backtestdata = pd.read_excel('./act.xlsx')

In [67]:
## keep
X_train = train[col]
X_test = test[col]
X_backtestdata = backtestdata[col]

Y_train = train['수익률(%)(2)']
Y_test = test['수익률(%)(2)']
Y_backtestdata = backtestdata['수익률(%)(2)']

model =XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=1.2, learning_rate=0.07,
       max_delta_step=0, max_depth=5, min_child_weight=1.0, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

model.fit(X_train[col] ,Y_train)
a = model.predict(X_test[col])
b = model.predict(X_backtestdata[col])
t = metrics.confusion_matrix(Y_backtestdata,b)
print('val: {x:.5}, acc: {y:.5}, pre:{g:.5}'.format(x = metrics.accuracy_score(a,Y_test), y = metrics.accuracy_score(b,Y_backtestdata),
                                                     g = t[1,1]/t[:,1].sum() ),'\n', metrics.confusion_matrix(Y_backtestdata,b))

val: 0.71667, acc: 0.625, pre:0.75 
 [[11  3]
 [ 9  9]]
